In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.22일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 8024 <= seq and seq <= 9024")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210922 | 순번 : 1 | start 한화비즈메트로 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 2 | start 한국산업기술시험원.디지털탑프라자 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 3 | start 구로디지털단지우체국 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 4 | start JNK디지털타워 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 5 | start 에이스하이엔드타워2차 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 6 | start 만민중앙교회 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 7 | start 에이스하이엔드타워1차 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 8 | start 개봉사거리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 9 | start 천왕이펜하우스6단지후문 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 10 | start 천왕이펜하우스6단지후문 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 11 | start 연지타운2단지 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 12 | start 연지타운2단지 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 13 | start 남부교도소 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 14 | start 남부교도소 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 15 | start 삼성부동산.천왕2지구 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 16 | start 작은굴다리 public_seo

조회날짜 : 20210922 | 순번 : 133 | start 신도림역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 134 | start 현대우성아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 135 | start 동아1차아파트102동 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 136 | start 동아1차아파트105동 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 137 | start 1호선구일역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 138 | start 금강수목원아파트정문 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 139 | start 고척근린시장 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 140 | start 구로시장.남구로시장입구 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 141 | start 구로주공.구일우성APT public_seoulbusstts
조회날짜 : 20210922 | 순번 : 142 | start 구로1동주민센터.연예인APT public_seoulbusstts
조회날짜 : 20210922 | 순번 : 143 | start 천왕이펜하우스5단지입구 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 144 | start 구일고교.구일초등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 145 | start 구로1동주민센터.연예인APT public_seoulbusstts
조회날짜 : 20210922 | 순번 : 146 | start 구로디지털단지역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 147 | start 구일초등학교.구일고등학교 public_seoulbusstts
조회날짜 : 

조회날짜 : 20210922 | 순번 : 261 | start 독산3동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 262 | start 독산3동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 263 | start 모두의학교.금천문화예술정보학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 264 | start 모두의학교.금천문화예술정보학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 265 | start 모두의학교.금천문화예술정보학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 266 | start 관악구보훈회관.신림푸르지오 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 267 | start 독산고등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 268 | start 독산고등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 269 | start 독산3치안센터.꿈꾸는나무 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 270 | start 독산3치안센터.꿈꾸는나무 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 271 | start 금천우체국.독산1동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 272 | start 진도아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 273 | start 제일모직아울렛 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 274 | start 벽산디지털밸리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 275 | start 마리오아울렛 public_seoulbusstts
조회날짜 

조회날짜 : 20210922 | 순번 : 390 | start 가산디지털단지역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 391 | start LG전자가산A public_seoulbusstts
조회날짜 : 20210922 | 순번 : 392 | start KCC웰츠밸리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 393 | start 독산고개입구 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 394 | start 벽산6단지입구 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 395 | start 독산역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 396 | start 독산역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 397 | start 남부여성발전센터앞 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 398 | start 남부여성발전센터앞 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 399 | start 산업단지백양사 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 400 | start 에이스하이엔드10차 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 401 | start 대륭테크노타운17차 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 402 | start 금천패션아울렛사거리.마리오 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 403 | start 범일운수종점 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 404 | start 한울중학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 405 | start 한울중학교 public_s

조회날짜 : 20210922 | 순번 : 521 | start 문래동남성아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 522 | start 문래동남성아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 523 | start 남부지방법원등기국.구로세무서(에이스하이테크시티) public_seoulbusstts
조회날짜 : 20210922 | 순번 : 524 | start 남부지방법원등기국.구로세무서(에이스하이테크시티) public_seoulbusstts
조회날짜 : 20210922 | 순번 : 525 | start 영등포역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 526 | start 영등포역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 527 | start 여의도환승센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 528 | start 여의도환승센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 529 | start 서울영림초등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 530 | start 강남성심병원.대림성모병원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 531 | start 경남아너스빌아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 532 | start 보라매공원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 533 | start 보라매역.동작세무서 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 534 | start 서울지방병무청 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 535 | start 여의도환승센터 public_seoulbusstts
조회날짜 : 2021

조회날짜 : 20210922 | 순번 : 652 | start 대윤병원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 653 | start 대신시장 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 654 | start 영등포농협.신길2동새마을금고 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 655 | start 우신초등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 656 | start 신길한화꿈에그린아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 657 | start 성락교회 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 658 | start 성락교회 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 659 | start 도림동자치회관 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 660 | start 우신초등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 661 | start 사러가쇼핑센타 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 662 | start 영등포농협.신길2동새마을금고 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 663 | start 신풍역.백악관웨딩문화원앞 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 664 | start 신길5동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 665 | start 신길5동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 666 | start 신풍역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 667 | start 신길5동새마을금고우

조회날짜 : 20210922 | 순번 : 783 | start 도림사거리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 784 | start 기림약국 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 785 | start 신길광장공원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 786 | start 코사마트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 787 | start 해오름식당 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 788 | start 우리은행 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 789 | start 두산위브진주아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 790 | start 현대3차아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 791 | start CM충무병원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 792 | start 대방역 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 793 | start 타임스퀘어 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 794 | start 당산신동아파밀리에 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 795 | start 영등포청과시장 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 796 | start 영등포청과시장 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 797 | start 서울시립청소년성문화센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 798 | start 서울시립청소년성문화센터 public_seoulbusstts

조회날짜 : 20210922 | 순번 : 915 | start 크라운사거리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 916 | start 도림동주민센터 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 917 | start 도림교회 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 918 | start 현대홈타운아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 919 | start 건영아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 920 | start 선유고등학교 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 921 | start 도림고가.동아에코빌 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 922 | start 거성현대6차아파트 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 923 | start 도림새마을금고 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 924 | start e편한세상문래에듀플라츠 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 925 | start SK엔카 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 926 | start 도림사거리 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 927 | start 대림요양병원 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 928 | start 선유도역.롯데홈쇼핑 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 929 | start 정현부페 public_seoulbusstts
조회날짜 : 20210922 | 순번 : 930 | start 당산초등학교 public_seoulbus